In [1]:
import pandas as pd
import os

In [2]:
import glob

def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:
                if "Bibliography" in file:
                    continue
                if("doclist" in file):
                    continue
                if(".docx" in file):
                    r.append(os.path.join(subdir, file))                                                                         
    return r                                                                                                          



In [3]:
file_list = list_files("/Users/Mihir/Downloads/Sample/")

In [4]:
import docx
import pandas as pd
words = []
#adding words to files list
for i in file_list:
    doc = docx.Document(i)
    words.append([p.text for p in doc.paragraphs])
xml = "/Users/Mihir/Downloads/Sample/wsj.xlsx"

#making each word in paragraph formation and removing all words that come before "Body", which tells us when the paragraph begins
for i in range(len(words)):
    words[i] = ' '.join(words[i])
    words[i] = words[i][words[0].index("Body"):]

In [5]:
from dateutil.parser import parse
from pandas import DataFrame
import re
one = []
two = [] 
#words to drop 
drop_words= {'military', 'strike', 'attack', 'syria', 'iran', 'pakistan'}
three = []
#split up files between dates
for i in words:
    date =parse(i[i.index("Load-Date:") + 11:i.index("End of Document")])
    if(date.year<=2012):
        one.append(i)
        for j in drop_words:
            if(j in i):
                one.pop()
                break
    elif(date.year<=2015):
        two.append(i)
        for j in drop_words:
            if(j in i):
                two.pop()
                break
    else:
        three.append(i)
        for j in drop_words:
            if(j in i):
                three.pop()
                break



In [10]:
import numpy
import math
from datetime import datetime
df = pd.read_excel('/Users/Mihir/Downloads/Sample/wsj.xlsx', engine='openpyxl') 
t = df['Content'].tolist()
df['Content'] = df.Content.astype(str)
print(len(one))
for x in df.iterrows():
    if(pd.isnull(x)):
        continue
    date =pd.Timestamp(x[1]['date'])
    if(date.year<=2012):
        one.append(x[1]['Content'])
        for j in drop_words:
            if(j in x[1]['Content']):
                one.pop()
                break
    elif(date.year<=2015):
        two.append(x[1]['Content'])
        for j in drop_words:
            if(j in x[1]['Content']):
                two.pop()
                break
    else:
        three.append(x[1]['Content'])
        for j in drop_words:
            if(j in x[1]['Content']):
                three.pop()
                break
        

84


In [14]:
#convert each of the date arrays into data frames
df_one = DataFrame(one, columns = ['text'])
df_two = DataFrame(two, columns = ['text'])
df_three = DataFrame(three, columns = ['text'])
df_one['processed'] = df_one['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_one['processed'] = df_one['text'].map(lambda x: x.lower())

df_two['processed'] = df_two['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_two['processed'] = df_two['text'].map(lambda x: x.lower())

df_three['processed'] = df_three['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df_three['processed'] = df_three['text'].map(lambda x: x.lower())

In [15]:
import gensim
from gensim.utils import simple_preprocess
#remove the punctuations and preprocess words 
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = df_one.processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])


['body', 'the', 'feature', 'directing', 'debut', 'of', 'the', 'gifted', 'actor', 'peter', 'riegert', 'king', 'of', 'the', 'corner', 'comes', 'from', 'fiction', 'collection', 'titled', 'bad', 'jews', 'and', 'other', 'stories', 'by', 'gerald', 'shapiro', 'something', 'about']


In [16]:
#convert to bigram/trigram model
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = " ".join(sent)
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [18]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_trigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

In [19]:
import nltk
words = []
#Lemmatize words and preprocess parargraphs for lda analysis
#
for i in range(len(data_words_bigrams)):
    wn = nltk.WordNetLemmatizer()
    words.append([wn.lemmatize(word) for word in data_words_bigrams[i]])

In [20]:
import re
import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.porter import PorterStemmer

In [21]:
from nltk.corpus import wordnet as wn
#define set of noun anv verbs
nouns = {x.name().split('.', 1)[0] for x in wn.all_synsets('n')}
verbs = {x.name().split('.', 1)[0] for x in wn.all_synsets('v')}

In [22]:
from nltk.corpus import wordnet
dictionary = {}
#create dictionary based on how many times a word appears in each of the files
for i in range(len(words)):
    s = set(words[i])
    for w in s:
        if(w in dictionary):
            dictionary[w]+=1
        else:
            dictionary[w] = 0
#filteer out words than are less than two letters long, appear in 85% of files, and by part of speach
for i in range(len(words)):
    temp = []
    for w in words[i]:
        temp.append(w)
    for w in words[i]:
        if(dictionary[w]>=(len(words) * 0.85) or len(w)<=2 or (w not in nouns) ):
            temp.remove(w)
    words[i] = temp
temp = []

In [23]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
#convert word to corpus to be read by LDA module
id2word = corpora.Dictionary(words)
texts = words
corpus = [id2word.doc2bow(text) for text in words]

In [24]:
from pprint import pprint
# number of topics
num_topics = 7
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"drone" + 0.008*"like" + 0.006*"song" + 0.005*"aircraft" + '
  '0.005*"year" + 0.004*"system" + 0.004*"official" + 0.004*"music" + '
  '0.004*"one" + 0.004*"ground"'),
 (1,
  '0.010*"drone" + 0.007*"air_force" + 0.006*"april" + 0.005*"year" + '
  '0.005*"like" + 0.005*"one" + 0.005*"rodeo" + 0.005*"pilot" + 0.004*"air" + '
  '0.004*"aircraft"'),
 (2,
  '0.031*"drone" + 0.007*"aircraft" + 0.006*"say" + 0.006*"june" + 0.005*"one" '
  '+ 0.004*"year" + 0.004*"system" + 0.004*"time" + 0.004*"company" + '
  '0.004*"section"'),
 (3,
  '0.016*"drone" + 0.006*"privacy" + 0.005*"time" + 0.005*"government" + '
  '0.005*"official" + 0.005*"surveillance" + 0.004*"one" + 0.004*"agency" + '
  '0.004*"like" + 0.004*"technology"'),
 (4,
  '0.021*"drone" + 0.009*"aircraft" + 0.007*"iraq" + 0.007*"official" + '
  '0.006*"air_force" + 0.006*"section" + 0.006*"say" + 0.006*"plane" + '
  '0.005*"department" + 0.005*"march"'),
 (5,
  '0.023*"drone" + 0.008*"aircraft" + 0.005*"system" + 0.005*"

In [25]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('/Users/Mihir/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '/Users/Mihir/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.003718  0.023331       1        1  23.120043
4     -0.028974  0.015203       2        1  17.546066
2     -0.025772 -0.040297       3        1  17.014535
5     -0.015934  0.012281       4        1  14.596703
6      0.018303  0.002922       5        1  11.528204
0      0.024384 -0.007800       6        1   9.852068
1      0.031711 -0.005641       7        1   6.342381, topic_info=          Term        Freq       Total Category  logprob  loglift
234  air_force   41.000000   41.000000  Default  30.0000  30.0000
150      drone  247.000000  247.000000  Default  29.0000  29.0000
66        like   47.000000   47.000000  Default  28.0000  28.0000
238      april   19.000000   19.000000  Default  27.0000  27.0000
205       song   34.000000   34.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
814  president    1.839849   30.913562   Topic7  -6.0721  -0.0636
226      world    1.801656   31.513427   Topic7  -6.0931  -0.1038
351       take    1.762841   26.938985   Topic7  -6.1149   0.0313
308   official    1.817627   58.047055   Topic7  -6.0843  -0.7058
280     ground    1.754369   30.529566   Topic7  -6.1197  -0.0987

[530 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
558       1  0.129017    ability
558       2  0.258034    ability
558       3  0.129017    ability
558       4  0.387052    ability
558       5  0.129017    ability
...     ...       ...        ...
919       2  0.330074  yesterday
919       3  0.066015  yesterday
919       4  0.198044  yesterday
919       5  0.132029  yesterday
2493      6  0.508958    ziegler

[1230 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 6, 7, 1, 2])

In [44]:
#Save LDA in file
df = LDAvis_prepared[1]
df = df.drop(columns = ["Freq", "Total", "logprob", "loglift"])
print(df)
compression_opts = dict(method='zip',
                         archive_name='fin.csv')  
df.to_csv('fin.zip', index=False,
           compression=compression_opts)  

           Term Category
143     company  Default
356    delivery  Default
341        year  Default
353  commercial  Default
419    aircraft  Default
..          ...      ...
113        time   Topic7
407  technology   Topic7
395        rule   Topic7
387       pilot   Topic7
609       small   Topic7

[580 rows x 2 columns]


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
df = LDAvis_prepared[1]
print(df)

           Term         Freq        Total Category  logprob  loglift
143     company  1102.000000  1102.000000  Default  30.0000  30.0000
356    delivery   389.000000   389.000000  Default  29.0000  29.0000
341        year   527.000000   527.000000  Default  28.0000  28.0000
353  commercial   400.000000   400.000000  Default  27.0000  27.0000
419    aircraft   480.000000   480.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
113        time    18.128053   443.182400   Topic7  -5.6544  -0.1972
407  technology    19.196894   641.863016   Topic7  -5.5971  -0.5104
395        rule    16.218605   411.177824   Topic7  -5.7657  -0.2336
387       pilot    16.805261   480.458013   Topic7  -5.7302  -0.3538
609       small    14.669851   313.587303   Topic7  -5.8661  -0.0630

[581 rows x 6 columns]


/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
